In [ ]:
import time
import codecs
import ephem
from ephem import degree
import datetime
from datetime import timedelta
import urllib.request
import csv

In [55]:
# Load the Space Catalogue as a dictionary
satcat = "https://celestrak.com/pub/satcat.txt"
sat_catalogue = urllib.request.urlopen(satcat).read().decode('utf-8').strip().split("\n")

sat_dict = {}
for x in sat_catalogue:
    try:
        sat_dict[x.split()[1]] = x
    except IndexError:
        continue

In [61]:
import json
import csv
import datetime
from datetime import timedelta
import ephem

# Open the local TLE data file
local_tle_file = "update_tle.tsv"
output_file = "output/data.json"

# Read the local TLE data file
with open(local_tle_file, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter='\t')
    sheetValues = list(reader)


In [63]:
print(sheetValues[1])

['1 00008U 58004  B 60095.16166626  .02607090 +00000-0 +00000-0 0  9993', '2 00008 065.0599 163.5585 0088318 135.6760 224.6847 16.28328133100004', 'n_00008']


In [79]:
data_list = []

for i in range(1, len(sheetValues)):
    row = sheetValues[i]
    tle_line0 = row[0]
    tle_line1 = row[1]
    norad_id = row[2].replace('n_', '')
    possible_satellite_reentry_date = sat_dict[norad_id][75:85] + " " + "00:00:00"
    
    print(norad_id)

    try:
        # Read TLE
        name = "sat_data"
        tle_rec = ephem.readtle(name, tle_line0, tle_line1)
        tle_rec.compute(possible_satellite_reentry_date)
        
        # Set reentry threshold to 122 km (122000 meters)
        reentry_threshold = 122000  # official entry interface by ESA and NASA
    
        # Check if the initial elevation is above or below the threshold
        if int(tle_rec.elevation) >= reentry_threshold:

            d = datetime.datetime.strptime(possible_satellite_reentry_date, '%Y-%m-%d %H:%M:%S')

            # Check whether the debris reenters after max two days 
            for i in range(0, 48):  # 48 hours
                parseHour = d + timedelta(hours=i)
                tle_rec.compute(parseHour)
                print(tle_rec.elevation)

                if int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry   
                    print("yee")

                    data_list.append({
                        "tle_0": tle_line0,
                        "tle_1": tle_line1,
                        "lat": tle_rec.sublat / ephem.degree,
                        "lon": tle_rec.sublong / ephem.degree,
                        "norad_cat_num": norad_id,
                        "satellite_name": sat_dict[norad_id][23:47].strip(),
                        "ownership": sat_dict[norad_id][49:54].strip(),
                        "launch_date": sat_dict[norad_id][56:66].strip(),
                        "satellite_decay": sat_dict[norad_id][75:85].strip(),
                        "satellite_decay_hour": str(parseHour),
                        "rcs": sat_dict[norad_id][119:127].strip()
                    })
                    
                    # Add further detailed tracking points
                    for offset_minutes in [2, 6]:
                        nextMinutes = parseHour + timedelta(minutes=offset_minutes)
                        tle_rec.compute(nextMinutes)
                        data_list.append({
                            "tle_0": tle_line0,
                            "tle_1": tle_line1,
                            "lat": tle_rec.sublat / ephem.degree,
                            "lon": tle_rec.sublong / ephem.degree,
                            "norad_cat_num": norad_id,
                            "satellite_name": sat_dict[norad_id][23:47].strip(),
                            "ownership": sat_dict[norad_id][49:54].strip(),
                            "launch_date": sat_dict[norad_id][56:66].strip(),
                            "satellite_decay": sat_dict[norad_id][75:85].strip(),
                            "satellite_decay_hour": str(nextMinutes),
                            "rcs": sat_dict[norad_id][119:127].strip()
                        })
                    break

        elif int(tle_rec.elevation) <= reentry_threshold:  # threshold of reentry

            for offset_minutes in [0, 2, 6]:
                nextMinutes = d + timedelta(minutes=offset_minutes)
                tle_rec.compute(nextMinutes)
                data_list.append({
                    "tle_0": tle_line0,
                    "tle_1": tle_line1,
                    "lat": tle_rec.sublat / ephem.degree,
                    "lon": tle_rec.sublong / ephem.degree,
                    "norad_cat_num": norad_id,
                    "satellite_name": sat_dict[norad_id][23:47].strip(),
                    "ownership": sat_dict[norad_id][49:54].strip(),
                    "launch_date": sat_dict[norad_id][56:66].strip(),
                    "satellite_decay": sat_dict[norad_id][75:85].strip(),
                    "satellite_decay_hour": str(nextMinutes),
                    "rcs": sat_dict[norad_id][119:127].strip()
                })
        break
    except Exception as e:
        print(e)
        continue

print("\nFinished parsing!")


00008
258931.375
205694.546875
yee

Finished parsing!


In [66]:
# Write data to minified JSON file
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, separators=(',', ':'))
